In [1]:
#imports
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from imblearn.over_sampling import SMOTE 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import keras
from keras.utils import normalize
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from keras.utils import to_categorical


Using TensorFlow backend.


In [4]:
#Models: Neural Net, SVM, Random Forest
#train equal amounts of fraud and not fraud data
#neural net: ? number of fully connected layers - softmax activation at the end, relu in the middle, adam optimizer, could use dropout, also use batch norm
#SVM: we can choose kernel/degree (3), slack/seed, penalized SVM 
#random forest: 

In [5]:
data = pd.read_csv('creditcard.csv')

In [6]:
data = data.drop('Time', 1)

In [7]:
data = data.as_matrix()

In [8]:
y = data[ : ,data.shape[1]-1:data.shape[1]]
X = data[:, 0:data.shape[1]-1]

In [9]:
#we dropped the time column 
#maybe time might matter - for mini rapid transactions?

In [10]:
X = preprocessing.scale(X) #perform feature normalization - subtract mean divide by standard deviation

In [11]:
# sm = SMOTE(random_state=42)
# X, y = sm.fit_sample(X, y.ravel())

In [12]:
#split using sklearn
X_train, X_test, y_train, y_test = train_test_split(X, y)
print(X_train.shape)
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_sample(X_train, y_train.ravel())
X_train.shape


(213605, 29)


(426458, 29)

In [13]:
#one-hot encode the y vectors:
y_train_cat = to_categorical(y_train, num_classes=2)

In [14]:
# split = int(X_train.shape[0]*.8)
# x_training = X_train[:split]
# x_validation = X_train[split:]
# y_training = y_train_cat[:split]
# y_validation = y_train_cat[split:]

In [15]:
#Neural Net - Murtaza
#SVM - Naren
#Random Forest - Rohan

In [16]:
model = Sequential()

In [17]:
model.add(Dense(16, input_shape=(29,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.3))

In [18]:
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.4))

In [19]:
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.4))

In [20]:
# model.add(Dense(128))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(.5))

In [21]:
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.4))

In [22]:
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.4))

In [23]:
model.add(Dense(16))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.3))

In [24]:
model.add(Dense(2))
model.add(BatchNormalization())
model.add(Activation('softmax'))

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                480       
_________________________________________________________________
batch_normalization_1 (Batch (None, 16)                64        
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                544       
_________________________________________________________________
batch_normalization_2 (Batch (None, 32)                128       
_________________________________________________________________
activation_2 (Activation)    (None, 32)                0         
__________

In [26]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# class_weights= {0:1, 1:100}

In [ ]:
model.fit(X_train, y_train_cat, batch_size=64, verbose=2, validation_split=.33, epochs=10)

Train on 285726 samples, validate on 140732 samples
Epoch 1/10
23s - loss: 0.1832 - acc: 0.9429 - val_loss: 0.3045 - val_acc: 0.8597
Epoch 2/10
21s - loss: 0.1113 - acc: 0.9615 - val_loss: 0.2675 - val_acc: 0.8591
Epoch 3/10


In [ ]:
def convertProbsToClass(y_scores):
    l = []
    for i in range(y_scores.shape[0]):
        val1 = y_scores[i][0]
        val2 = y_scores[i][1]
        if val1 >= val2:
            l.append(0)
        elif val2 > val1:
            l.append(1)
    return np.array(l) 

In [ ]:
# # y_scores = model.predict(x_validation)
# ys = convertProbsToClass(y_scores)
# y_actual = convertProbsToClass(y_validation)
# print(average_precision_score(y_actual, ys))

In [ ]:
#https://keras.io/getting-started/faq/ - for saving the model


In [ ]:
# from keras.models import load_model

# model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
# model = load_model('my_model.h5')

In [ ]:
y_scores = model.predict(X_test)
ys = convertProbsToClass(y_scores)
y_actual = y_test
print(average_precision_score(y_actual, ys))

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
target_names = ['class 0', 'class 1']
print(classification_report(y_actual, ys, target_names=target_names, digits=4))